In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [2]:
#导入数据
factor_df=pd.read_csv("/data01/data_for_intern/factor_data_13_19.csv")
factor_df.head()

,stock_code,trade_date,factor_0,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,...,factor_32,factor_33,factor_34,factor_35,factor_36,factor_37,factor_38,factor_39,factor_40,factor_41
0,000001.SZ,2013-01-04,0.007966,0.024510,0.032475,-8.0,-0.114170,0.03,0.440203,-0.376429,...,0.014641,0.790698,0.724551,0.812212,20.86980,43.5275,0.741100,0.680235,1.57469,3.43924
1,000001.SZ,2013-01-07,0.023154,0.006258,0.029412,-7.0,-0.589069,-0.31,0.129240,-0.840631,...,0.010109,0.856688,0.909091,0.841909,18.50720,41.6409,0.741100,0.704779,1.47261,3.15030
2,000001.SZ,2013-01-08,0.004295,0.026994,0.031288,-5.0,-0.150607,0.30,0.957488,-0.160806,...,0.007699,0.676724,0.727273,0.786972,20.08230,29.5337,0.605634,0.705782,1.48008,3.02077
3,000001.SZ,2013-01-09,0.003759,0.010025,0.013784,-3.0,-0.836842,0.14,0.802262,-0.532596,...,0.004680,0.632231,0.642424,0.759596,8.66294,27.9438,0.536232,0.699468,1.47922,3.01475
4,000001.SZ,2013-01-10,0.015132,0.004414,0.019546,-1.0,-0.565992,-0.01,0.452223,-0.712085,...,0.003882,0.629167,0.516667,0.628788,12.20690,27.2890,0.520000,0.690864,1.51058,3.06918


In [3]:
trading_df=pd.read_csv("/data01/data_for_intern/trading_data_13_19.csv")
trading_df.head()

,stock_code,trade_date,open,high,low,close,turnover_shares,turnover_value,trade_status,return_adj
0,000001.SZ,2013-01-04,16.32,16.45,15.92,15.99,44385100.0,717568000.0,1.0,-0.001873
1,000001.SZ,2013-01-07,15.98,16.35,15.88,16.30,35716900.0,578450000.0,1.0,0.019387
2,000001.SZ,2013-01-08,16.30,16.37,15.86,16.00,31247900.0,501360000.0,1.0,-0.018405
3,000001.SZ,2013-01-09,15.96,16.02,15.80,15.86,25132900.0,399696000.0,1.0,-0.008750
4,000001.SZ,2013-01-10,15.86,16.10,15.79,15.87,24003000.0,383348000.0,1.0,0.000631


In [4]:
valuation_df=pd.read_csv("/data01/data_for_intern/valuation_data_13_19.csv")
valuation_df.head()

,stock_code,trade_date,pe_ratio,turnover_ratio,pb_ratio,ps_ratio,pcf_ratio,capitalization,market_cap,circulating_cap,circulating_market_cap,pe_ratio_lyr
0,000001.SZ,2013-01-04,6.3855,1.4293,1.0087,2.1293,1.5452,512335.0,819.224,310536.0,496.547,7.9702
1,000001.SZ,2013-01-07,6.5093,1.1502,1.0283,2.1706,1.5752,512335.0,835.106,310536.0,506.173,8.1247
2,000001.SZ,2013-01-08,6.3895,1.0063,1.0094,2.1307,1.5462,512335.0,819.736,310536.0,496.857,7.9751
3,000001.SZ,2013-01-09,6.3336,0.8093,1.0005,2.1120,1.5326,512335.0,812.563,310536.0,492.510,7.9054
4,000001.SZ,2013-01-10,6.3376,0.7730,1.0012,2.1133,1.5336,512335.0,813.076,310536.0,492.820,7.9104


In [5]:
#label
return_df=pd.read_csv("/data01/data_for_intern/return_data_13_19.csv")
return_df.head()

,stock_code,trade_date,return_adj_d001,return_adj_d005,car_hs300_b30_d001,car_hs300_b30_d005
0,000001.SZ,2013-01-04,0.019387,-0.028143,0.013050,-0.005601
1,000001.SZ,2013-01-07,-0.018405,0.048466,-0.012566,0.025594
2,000001.SZ,2013-01-08,-0.008750,0.100625,-0.009196,0.061081
3,000001.SZ,2013-01-09,0.000631,0.123581,-0.001883,0.094744
4,000001.SZ,2013-01-10,-0.020794,0.117832,0.005961,0.105292


In [6]:
#以car_hs300_b30_d001为label
return_label=return_df.drop(columns=['return_adj_d001','return_adj_d005','car_hs300_b30_d005'])
return_label['car_hs300_b30_d001']=(return_label['car_hs300_b30_d001']>=0)*1
return_label

,stock_code,trade_date,car_hs300_b30_d001
0,000001.SZ,2013-01-04,1
1,000001.SZ,2013-01-07,0
2,000001.SZ,2013-01-08,0
3,000001.SZ,2013-01-09,0
4,000001.SZ,2013-01-10,1
...,...,...,...
5156943,603999.SH,2019-12-25,1
5156944,603999.SH,2019-12-26,0
5156945,603999.SH,2019-12-27,0
5156946,603999.SH,2019-12-30,0


In [7]:
#后续会用到的两个df是return_label和factor_df,但是两个df长度不同，故有问题，需要进一步清洗数据。
return_label[return_label['stock_code'].str.contains('SZ')]
#'SZ'股票有300820支，'SH'股票有603999支

,stock_code,trade_date,car_hs300_b30_d001
0,000001.SZ,2013-01-04,1
1,000001.SZ,2013-01-07,0
2,000001.SZ,2013-01-08,0
3,000001.SZ,2013-01-09,0
4,000001.SZ,2013-01-10,1
...,...,...,...
3136174,300820.SZ,2019-12-25,0
3136175,300820.SZ,2019-12-26,0
3136176,300820.SZ,2019-12-27,0
3136177,300820.SZ,2019-12-30,0


In [8]:
#利用内连接方式合并数据
merged_factor_return = factor_df.merge(return_label,on=["stock_code","trade_date"],how="inner")
merged_factor_return

,stock_code,trade_date,factor_0,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,...,factor_33,factor_34,factor_35,factor_36,factor_37,factor_38,factor_39,factor_40,factor_41,car_hs300_b30_d001
0,000001.SZ,2013-01-04,0.007966,0.024510,0.032475,-8.0,-0.114170,0.03,0.440203,-0.376429,...,0.790698,0.724551,0.812212,20.869800,43.5275,0.741100,0.680235,1.57469,3.43924,1
1,000001.SZ,2013-01-07,0.023154,0.006258,0.029412,-7.0,-0.589069,-0.31,0.129240,-0.840631,...,0.856688,0.909091,0.841909,18.507200,41.6409,0.741100,0.704779,1.47261,3.15030,0
2,000001.SZ,2013-01-08,0.004295,0.026994,0.031288,-5.0,-0.150607,0.30,0.957488,-0.160806,...,0.676724,0.727273,0.786972,20.082300,29.5337,0.605634,0.705782,1.48008,3.02077,0
3,000001.SZ,2013-01-09,0.003759,0.010025,0.013784,-3.0,-0.836842,0.14,0.802262,-0.532596,...,0.632231,0.642424,0.759596,8.662940,27.9438,0.536232,0.699468,1.47922,3.01475,0
4,000001.SZ,2013-01-10,0.015132,0.004414,0.019546,-1.0,-0.565992,-0.01,0.452223,-0.712085,...,0.629167,0.516667,0.628788,12.206900,27.2890,0.520000,0.690864,1.51058,3.06918,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5088415,603999.SH,2019-12-13,0.063063,0.007508,0.070571,-9.0,-0.875646,-0.32,0.138746,-0.876772,...,0.542857,0.855769,0.822900,1.148900,20.9302,0.029940,0.307711,1.14262,3.09775,1
5088416,603999.SH,2019-12-16,0.038682,0.007163,0.045845,-9.0,-0.880784,-0.18,0.306574,-0.751634,...,0.583333,0.884298,0.811730,0.782227,27.0718,0.187879,0.276938,1.04110,2.77682,0
5088417,603999.SH,2019-12-17,0.002821,0.016925,0.019746,-8.0,-0.854110,0.11,0.863933,-0.479172,...,0.654971,0.778761,0.839609,0.342224,29.0801,0.298013,0.260858,1.04584,2.66659,0
5088418,603999.SH,2019-12-18,0.010029,0.012894,0.022923,-7.0,-0.733061,0.04,0.511944,-0.716503,...,0.670659,0.736364,0.799807,0.391114,29.2537,0.324324,0.252109,1.07986,2.67018,1


In [9]:
#示范如何将数据转化为需要的格式
a=np.array([[1,2,3],[4,5,6],[7,8,9],[5,3,2]])
a,a.shape
b=np.zeros([2,3,2,2])
b[0,1,:,:]=a[:,1].reshape([1,2,2])
b

array([[[[0., 0.],
         [0., 0.]],

        [[2., 5.],
         [8., 3.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]])

In [10]:
#示范如何将数据转化为需要的格式
m=np.random.rand(2,4,3)
m

array([[[0.10959528, 0.98600974, 0.21858173],
        [0.08074427, 0.83746601, 0.54561489],
        [0.51147645, 0.97641632, 0.00673172],
        [0.30253163, 0.20257503, 0.68099054]],

       [[0.43941727, 0.17291169, 0.15711139],
        [0.26795916, 0.57026654, 0.99682051],
        [0.7174135 , 0.5821118 , 0.54243742],
        [0.87730415, 0.75820971, 0.51672239]]])

In [11]:
#示范如何将数据转化为需要的格式
d={}
b=np.zeros([2,3,2,2])
for i in range(2):
    for k in range(3):
        b[i,k,:,:]=m[i,:,k].reshape(1,2,2)
b

array([[[[0.10959528, 0.08074427],
         [0.51147645, 0.30253163]],

        [[0.98600974, 0.83746601],
         [0.97641632, 0.20257503]],

        [[0.21858173, 0.54561489],
         [0.00673172, 0.68099054]]],


       [[[0.43941727, 0.26795916],
         [0.7174135 , 0.87730415]],

        [[0.17291169, 0.57026654],
         [0.5821118 , 0.75820971]],

        [[0.15711139, 0.99682051],
         [0.54243742, 0.51672239]]]])

In [12]:
#举例
temp=pd.DataFrame(merged_factor_return.loc[0:63]).drop(['stock_code','trade_date','car_hs300_b30_d001'],axis=1)
temp

,factor_0,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,factor_8,factor_9,...,factor_32,factor_33,factor_34,factor_35,factor_36,factor_37,factor_38,factor_39,factor_40,factor_41
0,0.007966,0.024510,0.032475,-8.0,-0.114170,0.03,0.440203,-0.376429,-3.43,0.716400,...,0.014641,0.790698,0.724551,0.812212,20.86980,43.52750,0.741100,0.680235,1.574690,3.43924
1,0.023154,0.006258,0.029412,-7.0,-0.589069,-0.31,0.129240,-0.840631,-3.74,0.315400,...,0.010109,0.856688,0.909091,0.841909,18.50720,41.64090,0.741100,0.704779,1.472610,3.15030
2,0.004295,0.026994,0.031288,-5.0,-0.150607,0.30,0.957488,-0.160806,-3.44,0.961388,...,0.007699,0.676724,0.727273,0.786972,20.08230,29.53370,0.605634,0.705782,1.480080,3.02077
3,0.003759,0.010025,0.013784,-3.0,-0.836842,0.14,0.802262,-0.532596,-3.30,0.695788,...,0.004680,0.632231,0.642424,0.759596,8.66294,27.94380,0.536232,0.699468,1.479220,3.01475
4,0.015132,0.004414,0.019546,-1.0,-0.565992,-0.01,0.452223,-0.712085,-3.31,0.601559,...,0.003882,0.629167,0.516667,0.628788,12.20690,27.28900,0.520000,0.690864,1.510580,3.06918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.015664,0.010611,0.026276,-5.0,-0.138518,0.52,0.966731,-0.709321,0.46,0.562257,...,-0.005113,0.332842,0.094595,0.178967,34.25800,18.44740,0.280130,0.128938,0.541923,1.23431
60,0.013507,0.004002,0.017509,-3.0,-0.156744,-0.01,0.705837,-0.515211,0.50,0.847530,...,-0.003836,0.381513,0.116505,0.144542,13.78190,20.89950,0.247619,0.145760,0.495528,1.37526
61,0.001006,0.037726,0.038732,-1.0,-0.054678,0.54,0.955270,-0.250000,1.05,0.928821,...,-0.005841,0.334385,0.076271,0.095790,31.89540,20.68060,0.314038,0.165748,0.497001,1.35652
62,0.005615,0.015314,0.020929,-2.0,-0.071284,-0.01,0.271295,-0.706951,0.52,0.589265,...,-0.002679,0.111579,0.149733,0.114170,16.14460,8.30000,0.690875,0.214199,0.597747,1.80500


In [13]:
#举例
temp=np.array(temp.values).reshape(1,64,42)
train_data=np.zeros([1,42,8,8])
for i in range(1):
    for k in range(42):
        train_data[i,k,:,:]=temp[i,:,k].reshape(1,8,8)
train_data

array([[[[ 7.96575e-03,  2.31540e-02,  4.29458e-03, ...,  1.07121e-02,
           1.03292e-01,  4.67289e-02],
         [ 1.70357e-02,  3.27684e-02,  3.07778e-02, ...,  1.99900e-02,
           2.74588e-02,  4.10256e-03],
         [ 9.96868e-02,  1.17371e-02,  1.13744e-02, ...,  2.38096e-02,
           1.93192e-02,  1.29032e-02],
         ...,
         [ 7.56304e-03,  1.14437e-02,  8.40710e-03, ...,  4.47830e-02,
           1.38248e-02,  2.01878e-02],
         [ 8.58609e-02,  9.56935e-03,  8.27881e-03, ...,  2.25530e-02,
           0.00000e+00,  1.74825e-02],
         [ 1.98020e-02,  2.55152e-02,  1.11868e-02, ...,  1.00606e-03,
           5.61514e-03,  1.54235e-03]],

        [[ 2.45098e-02,  6.25779e-03,  2.69938e-02, ...,  2.64650e-02,
           0.00000e+00,  1.75238e-03],
         [ 2.21466e-02,  1.35594e-02,  1.51091e-02, ...,  3.54823e-02,
           3.64454e-02,  2.87179e-02],
         [ 0.00000e+00,  3.14554e-02,  1.42181e-02, ...,  2.47080e-02,
           2.52990e-02,  1.15207e

In [14]:
#dataframe预处理
df=pd.DataFrame(merged_factor_return).drop(['trade_date','car_hs300_b30_d001'],axis=1)

In [15]:
#筛选股票名称
name=np.unique(merged_factor_return['stock_code'].values)

In [16]:
#整理训练数据
waste=[]
for idx in name:
    if idx=='600684.SH':#留320-64天记录作为valid_data和test_data
        break
    temp_df=df[df['stock_code']==idx]
    bound=temp_df.shape[0]-320 #留320-64天记录作为valid_data和test_data
    if bound<0:
        waste.append(idx)
        continue
    temp_df=temp_df.drop(['stock_code'],axis=1)
    for i in range(0,bound,64):
        temp=temp_df.iloc[i:i+64]
        temp=np.array(temp.values).reshape(1,64,42)
        temp_mat=np.zeros([1,42,8,8])
        for k in range(42):
            temp_mat[0,k,:,:]=temp[0,:,k].reshape(1,8,8)
        train_data=np.concatenate((train_data,temp_mat),axis=0)
#时间过长来不及做完完整的循环

In [17]:
train_data.shape

(51146, 42, 8, 8)

In [18]:
idx

'600684.SH'

In [19]:
backup_train_data=train_data

In [20]:
waste

['000522.SZ',
 '000527.SZ',
 '000602.SZ',
 '001914.SZ',
 '002933.SZ',
 '002935.SZ',
 '002936.SZ',
 '002937.SZ',
 '002938.SZ',
 '002939.SZ',
 '002940.SZ',
 '002941.SZ',
 '002942.SZ',
 '002943.SZ',
 '002945.SZ',
 '002946.SZ',
 '002947.SZ',
 '002948.SZ',
 '002949.SZ',
 '002950.SZ',
 '002951.SZ',
 '002952.SZ',
 '002953.SZ',
 '002955.SZ',
 '002956.SZ',
 '002957.SZ',
 '002958.SZ',
 '002959.SZ',
 '002960.SZ',
 '002961.SZ',
 '002962.SZ',
 '002963.SZ',
 '002965.SZ',
 '002966.SZ',
 '002967.SZ',
 '002968.SZ',
 '002969.SZ',
 '002970.SZ',
 '003816.SZ',
 '300564.SZ',
 '300594.SZ',
 '300674.SZ',
 '300694.SZ',
 '300748.SZ',
 '300749.SZ',
 '300751.SZ',
 '300752.SZ',
 '300753.SZ',
 '300755.SZ',
 '300756.SZ',
 '300757.SZ',
 '300758.SZ',
 '300759.SZ',
 '300760.SZ',
 '300761.SZ',
 '300762.SZ',
 '300763.SZ',
 '300765.SZ',
 '300766.SZ',
 '300767.SZ',
 '300768.SZ',
 '300769.SZ',
 '300770.SZ',
 '300771.SZ',
 '300772.SZ',
 '300773.SZ',
 '300775.SZ',
 '300776.SZ',
 '300777.SZ',
 '300778.SZ',
 '300779.SZ',
 '3007

In [21]:
type(name)

numpy.ndarray

In [22]:
clean_name = np.setdiff1d(name,waste)
clean_name

array(['000001.SZ', '000002.SZ', '000004.SZ', ..., '603997.SH',
       '603998.SH', '603999.SH'], dtype=object)

In [23]:
#整理valid数据
temp=pd.DataFrame(merged_factor_return.loc[1694-320:1437]).drop(['stock_code','trade_date','car_hs300_b30_d001'],axis=1)
temp=np.array(temp.values).reshape(1,64,42)
valid_data=np.zeros([1,42,8,8])
for i in range(1):
    for k in range(42):
        valid_data[i,k,:,:]=temp[i,:,k].reshape(1,8,8)

In [24]:
valid_data.shape

(1, 42, 8, 8)

In [25]:
for idx in clean_name:
    if idx=='600684.SH':
        break
    temp_df=df[df['stock_code']==idx]
    bound=temp_df.shape[0]-320 #留320-64天记录作为valid_data和test_data
    temp_df=temp_df.drop(['stock_code'],axis=1)
    for i in range(bound,bound+128,64):
        temp=temp_df.iloc[i:i+64]
        temp=np.array(temp.values).reshape(1,64,42)
        temp_mat=np.zeros([1,42,8,8])
        for k in range(42):
            temp_mat[0,k,:,:]=temp[0,:,k].reshape(1,8,8)
        valid_data=np.concatenate((valid_data,temp_mat),axis=0)

In [26]:
valid_data.shape

(5371, 42, 8, 8)

In [27]:
backup_valid_data=valid_data

In [28]:
#整理train_label
train_label=np.zeros(1,)
train_label[0]=merged_factor_return['car_hs300_b30_d001'].loc[64]
waste_sec=[]
for idx in name:
    if idx=='600684.SH':
        break
    temp_df=merged_factor_return[merged_factor_return['stock_code']==idx]
    bound=temp_df.shape[0]-320 
    if bound<0:
        waste_sec.append(idx)
        continue
    for i in range(64,bound+64,64):
        temp=np.array(temp_df['car_hs300_b30_d001'].iloc[i])
        train_label=np.concatenate((train_label,temp.reshape(1,)),axis=0)

In [29]:
backup_train_label=train_label

In [30]:
train_label.shape

(51146,)

In [31]:
train_data=train_data[0:51146,:,:,:]

In [32]:
#整理valid_label
valid_label=np.zeros(1,)
valid_label[0]=merged_factor_return['car_hs300_b30_d001'].loc[1438]
for idx in clean_name:
    if idx=='600684.SH':
        break
    temp_df=merged_factor_return[merged_factor_return['stock_code']==idx]
    bound=temp_df.shape[0]-320 
    for i in range(bound+64,bound+192,64):
        temp=np.array(temp_df['car_hs300_b30_d001'].iloc[i])
        valid_label=np.concatenate((valid_label,temp.reshape(1,)),axis=0)

In [33]:
valid_label.shape

(5371,)

In [36]:
np.save('train_data',train_data)
np.save('valid_data',valid_data)
np.save('train_label',train_label)
np.save('valid_label',valid_label)#np.load(fname)读取

In [37]:
#搭建模型
class BasicBlock(nn.Module):#基本残差网络的一个模块类
    expansion = 1#每一个residual block中不改变width,height,channel数，即增加的residual部分不需要做卷积处理

    def __init__(self, in_channel, out_channel, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channel, out_channels=out_channel,
                               kernel_size=3, stride=stride, padding=1, bias=False)#stride=1,kernel_size=3,padding=1保证了data的
                                                                                   #size不变 
        self.bn1 = nn.BatchNorm2d(out_channel)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=out_channel, out_channels=out_channel,
                               kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channel)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        if self.downsample is not None:  #BasicBlock内不需要调整residual的height,width,channel
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += identity
        out = self.relu(out)

        return out

In [38]:
#组建block成ResNet
class ResNet(nn.Module):

    def __init__(self, block, blocks_num, num_classes=1000, feature_channel=3,device = torch.device("cpu")):
        super(ResNet, self).__init__()
        self.in_channel = 64

        self.conv1 = nn.Conv2d(in_channels=feature_channel, out_channels=self.in_channel, kernel_size=2, stride=2,
                               padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_channel)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)  #channel数变为n/2(非整数时向下取整)
        self.layer1 = self._make_layer(block, 64, blocks_num[0])         #按照已有结论按二次方形式增长ResNet不同阶段的channel
        self.layer2 = self._make_layer(block, 128, blocks_num[1], stride=2)#stride=2表示想把上一个layer传过来的size缩减为1/2
        self.layer3 = self._make_layer(block, 256, blocks_num[2], stride=2)
        self.layer4 = self._make_layer(block, 512, blocks_num[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # output size = (1, 1) 
        self.fc = nn.Linear(512 * block.expansion, num_classes) #fully connected

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight.to(device), mode='fan_out', nonlinearity='relu')

    def _make_layer(self, block, channel, block_num, stride=1):
        downsample = None
        if stride != 1 or self.in_channel != channel * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channel, channel * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(channel * block.expansion))

        layers = []
        layers.append(block(self.in_channel, channel, downsample=downsample, stride=stride))
        self.in_channel = channel * block.expansion

        for _ in range(1, block_num):
            layers.append(block(self.in_channel, channel))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x) #预处理
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x) #残差网络
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)#fully connection
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

In [39]:
def resnet34(num_classes=1000,feature_channel=3,device = torch.device("cpu")):
    return ResNet(BasicBlock, [3, 4, 6, 3], num_classes=num_classes, feature_channel=feature_channel,device = torch.device("cpu"))

In [52]:
from torch.utils.data import Dataset
#实现custom pytorch dataset
class GetLoader(Dataset):
# 初始化函数，得到数据
    def __init__(self, data_root, data_label):
        self.data = data_root
        self.label = data_label
    # index是根据batchsize划分数据后得到的索引，最后将data和对应的labels进行一起返回
    def __getitem__(self, index):
        data = self.data[index]
        labels = self.label[index]
        return data, labels
    # 该函数返回数据大小长度，目的是DataLoader方便划分，如果不知道大小，DataLoader会一执行错误
    def __len__(self):
        return len(self.data)


In [133]:
#清理数据中的nan
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
def fill_ndarray(train_data):
    for k in range(train_data.shape[0]):
        for i in range(train_data.shape[1]):
            train_data[k,i,:,:]=my_imputer.fit_transform(train_data[k,i,:,:])
        return train_data

In [144]:
train_data=fill_ndarray(train_data)
train_data[np.isnan(train_data)]=0

In [145]:
valid_data=fill_ndarray(valid_data)
valid_data[np.isnan(valid_data)]=0

In [147]:
train=GetLoader(train_data,train_label)
valid=GetLoader(valid_data,valid_label)

In [148]:
train_label.shape

(51146,)

In [149]:
train_data.shape

(51146, 42, 8, 8)

In [183]:
#device=torch.device('cuda'if torch.cuda.is_available() else 'cpu')
#Hyper parameters
num_epochs=200
num_classes=2
batch_size=512
learning_rate=0.001

In [184]:
train_loader=torch.utils.data.DataLoader(dataset=train,batch_size=batch_size,shuffle=True,num_workers=1)
valid_loader=torch.utils.data.DataLoader(dataset=valid,batch_size=5371,shuffle=False,num_workers=1)

In [185]:
device = torch.device("cuda:1")
model=resnet34(num_classes=num_classes,feature_channel=42,device=device)
model.to(device)
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
total_step=0
for epoch in range(num_epochs):
    for data in train_loader:
        images, labels = data
        images=images.type(torch.FloatTensor)
        labels=labels.type(torch.FloatTensor)
        images=images.to(device)
        labels=labels.to(device)
        #forward pass
        outputs=model(images)
        loss=loss_function(outputs,labels.long())
        
        #Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_step+=1
        if (total_step)%100==0:#each 10 iterations is one epoch
            print("Epoch [{}/{}],step[{}] Loss:{:.4f}".format(epoch+1,num_epochs,total_step,loss.item()))

Epoch [1/200],step[100] Loss:0.6744
Epoch [2/200],step[200] Loss:0.6647
Epoch [3/200],step[300] Loss:0.6459
Epoch [4/200],step[400] Loss:0.6428
Epoch [5/200],step[500] Loss:0.6580
Epoch [6/200],step[600] Loss:0.6417
Epoch [7/200],step[700] Loss:0.6423
Epoch [8/200],step[800] Loss:0.6659
Epoch [9/200],step[900] Loss:0.6013
Epoch [10/200],step[1000] Loss:0.6487
Epoch [11/200],step[1100] Loss:0.6080
Epoch [12/200],step[1200] Loss:0.5922
Epoch [13/200],step[1300] Loss:0.6201
Epoch [14/200],step[1400] Loss:0.5829
Epoch [15/200],step[1500] Loss:0.6122
Epoch [16/200],step[1600] Loss:0.5951
Epoch [17/200],step[1700] Loss:0.5459
Epoch [18/200],step[1800] Loss:0.5946
Epoch [19/200],step[1900] Loss:0.5846
Epoch [20/200],step[2000] Loss:0.5474
Epoch [21/200],step[2100] Loss:0.5353
Epoch [22/200],step[2200] Loss:0.5097
Epoch [23/200],step[2300] Loss:0.4479
Epoch [24/200],step[2400] Loss:0.4493
Epoch [25/200],step[2500] Loss:0.4604
Epoch [26/200],step[2600] Loss:0.4766
Epoch [27/200],step[2700] Loss

In [186]:
with torch.no_grad():
    correct=0
    total=0
    for images, labels in valid_loader:
        images=images.type(torch.FloatTensor)
        labels=labels.type(torch.FloatTensor)
        images=images.to(device)
        labels=labels.to(device)
        outputs=model(images)
        _,predicted=torch.max(outputs.data,1)
        total+=labels.size(0)
        correct+=(predicted==labels).sum().item()
print('accuracy of the model on the test images: {}%'.format(100*correct/total))

accuracy of the model on the test images: 51.51740830385403%


In [180]:
predicted.shape

torch.Size([5371])

In [ ]:
#将数据带入模型
import ResNet
from ResNet import GetLoader,resnet34,model_optimize,model_evaluation,predict

In [ ]:
#设置模型参数
batch_size=256
learning_rate=0.001
num_classes=2
feature_channel=42
device = torch.device("cuda:1")

In [ ]:
#数据处理
train_=GetLoader(train_data,train_label)
valid_=GetLoader(valid_data,valid_label)
train_loader=torch.utils.data.DataLoader(dataset=train_,batch_size=batch_size,shuffle=True,num_workers=1)
valid_loader=torch.utils.data.DataLoader(dataset=valid_,batch_size=batch_size,shuffle=False,num_workers=1)

In [ ]:
model=resnet34(num_classes=num_classes,feature_channel=feature_channel,device = device)
model.to(device)
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [ ]:
model_optimize(model,num_epochs,train_loader,device,loss_function,optimizer)

In [ ]:
model_evaluation(valid_loader)